# Explore here

In [2]:
import pandas as pd
from pathlib import Path
import zipfile, os
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

import warnings
import joblib
warnings.filterwarnings('ignore')

In [3]:
#Ubicamos modelo
#Indicamos cuál es la carpeta en directorio (src)
BASE_DIR = Path().resolve()

#Indicamos la ruta del modelo
MODEL_PATH = BASE_DIR.parent / "models" / "xgb_model.pkl"

print(MODEL_PATH)
print(MODEL_PATH.exists())  #Comprobamos que el modelo existe = true

/workspaces/equipoEnergias_XGB_con_Streamlit/models/xgb_model.pkl
True


In [4]:
#Cargamos modelo
MODEL_PATH = "../models/xgb_model.pkl"
model = joblib.load(MODEL_PATH)
print("El modelo se ha cargado correctamente ✅")

El modelo se ha cargado correctamente ✅


In [5]:
#Sacamos las categorias usadas del modelo
print(model.feature_names_in_)

['demanda_lag_1' 'demanda_lag_24' 'demanda_lag_168' 'media_movil_24h'
 'hora' 'mes' 'es_finde' 'dia_semana' 'Madrid_temperature_2m'
 'Valencia_temperature_2m' 'Pais_Vasco_temperature_2m'
 'Cataluna_temperature_2m' 'Andalucia_temperature_2m']


In [7]:
#Probamos a predecir
X_test = pd.DataFrame([{
    "demanda_lag_1": 28000,
    "demanda_lag_24": 27500,
    "demanda_lag_168": 26000,
    "media_movil_24h": 27000,
    "hora": 18,
    "mes": 1,
    "es_finde": 0,
    "dia_semana": 2,
    "Madrid_temperature_2m": 30.5,
    "Valencia_temperature_2m": 29.0,
    "Pais_Vasco_temperature_2m": 22.0,
    "Cataluna_temperature_2m": 28.0,
    "Andalucia_temperature_2m": 33.0
}])

X_test = X_test[model.feature_names_in_]
pred = model.predict(X_test)
print("Predicción de demanda eléctrica (MW):", pred[0])

Predicción de demanda eléctrica (MW): 28347.047


In [8]:
#para el error de DATABASE_URL
def db_connect():
    db_url = os.getenv("DATABASE_URL")

    if not db_url:
        return None  # ← evita el crash

    engine = create_engine(db_url)
    return engine

In [10]:
print("Columnas que espera el modelo:")
print(model.feature_names_in_)

Columnas que espera el modelo:
['demanda_lag_1' 'demanda_lag_24' 'demanda_lag_168' 'media_movil_24h'
 'hora' 'mes' 'es_finde' 'dia_semana' 'Madrid_temperature_2m'
 'Valencia_temperature_2m' 'Pais_Vasco_temperature_2m'
 'Cataluna_temperature_2m' 'Andalucia_temperature_2m']


# -----------------------------
# Slider interactivo de hora con color fijo naranja eléctrico y emoji
# -----------------------------

# Columnas para controlar ancho (2/3 del ancho de pantalla)
col1, col2 = st.columns([2,1])

with col1:
    hora_real = st.slider(
        "Hora del día",
        min_value=0,
        max_value=23,
        value=18,  # valor por defecto = 6 PM
        step=1
    )

# Emoji dinámico según día/noche
if 6 <= hora_real <= 18:
    icono = "☀️"
else:
    icono = "🌙"

# Color fijo del slider: Naranja eléctrico (#FF6F00)
st.markdown(f"""
<style>
div[data-baseweb="slider"] input[type="range"] {{
    accent-color: #FF6F00;
}}
</style>
""", unsafe_allow_html=True)

# Mostrar la hora seleccionada con emoji
st.markdown(f"**Hora seleccionada:** {hora_real}h {icono}")



# Lista de nombres de días, ahora 1=Lunes ... 7=Domingo
dias_semana_nombres = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes", "Sábado", "Domingo"]

# Slider para el día de la semana (1 a 7)
dia_semana = st.slider(
    "Día de la semana",
    min_value=1,
    max_value=7,
    value=3,  # Por defecto Miércoles
    step=1
)

# Nombre del día seleccionado (ajustamos el índice -1)
dia_nombre = dias_semana_nombres[dia_semana - 1]

# Calcular si es fin de semana
es_finde_num = 1 if dia_semana in [6, 7] else 0
es_finde_texto = "Sí" if es_finde_num == 1 else "No"

# Mostrar información
st.markdown(f"**Día seleccionado:** {dia_nombre}")
st.markdown(f"**Es fin de semana:** {es_finde_texto}")

# -----------------------------
# MES + ESTACIÓN DEL AÑO
# -----------------------------

# Mapeo de meses
meses = {
    "Enero": 1,
    "Febrero": 2,
    "Marzo": 3,
    "Abril": 4,
    "Mayo": 5,
    "Junio": 6,
    "Julio": 7,
    "Agosto": 8,
    "Septiembre": 9,
    "Octubre": 10,
    "Noviembre": 11,
    "Diciembre": 12
}

# Selector de mes (menú desplegable)
mes_nombre = st.selectbox("Mes", list(meses.keys()))
mes = meses[mes_nombre]  # valor numérico para el modelo

# Cálculo de estación por mes completo
if mes in [12, 1, 2]:
    estacion = "❄️ Invierno"
elif mes in [3, 4, 5]:
    estacion = "🌱 Primavera"
elif mes in [6, 7, 8]:
    estacion = "☀️ Verano"
else:
    estacion = "🍂 Otoño"

# Mostrar estación
st.markdown(f"**Estación del año:** {estacion}")